In [1]:
!pwd

/content


In [2]:
# Declaring the working library
import os
os.chdir('/content')

In [3]:
# Installing the required library
# !pip install -U scikit-learn

In [4]:
import sklearn
print(sklearn.__version__)

1.3.2


__Evaluation Module__

In [5]:
import pandas as pd
import numpy as np
from sklearn import metrics

In [6]:
import numpy as np
from sklearn.metrics import mean_squared_log_error, r2_score

EPSILON = 1e-10


def _error(actual: np.ndarray, predicted: np.ndarray):
    """ Simple error """
    return actual - predicted


def _percentage_error(actual: np.ndarray, predicted: np.ndarray):
    """
    Percentage error
    Note: result is NOT multiplied by 100
    """
    return _error(actual, predicted) / (actual + EPSILON)


def _naive_forecasting(actual: np.ndarray, seasonality: int = 1):
    """ Naive forecasting method which just repeats previous samples """
    return actual[:-seasonality]


def _relative_error(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Relative Error """
    if benchmark is None or isinstance(benchmark, int):
        # If no benchmark prediction provided - use naive forecasting
        if not isinstance(benchmark, int):
            seasonality = 1
        else:
            seasonality = benchmark
        return _error(actual[seasonality:], predicted[seasonality:]) /\
               (_error(actual[seasonality:], _naive_forecasting(actual, seasonality)) + EPSILON)

    return _error(actual, predicted) / (_error(actual, benchmark) + EPSILON)


def _bounded_relative_error(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Bounded Relative Error """
    if benchmark is None or isinstance(benchmark, int):
        # If no benchmark prediction provided - use naive forecasting
        if not isinstance(benchmark, int):
            seasonality = 1
        else:
            seasonality = benchmark

        abs_err = np.abs(_error(actual[seasonality:], predicted[seasonality:]))
        abs_err_bench = np.abs(_error(actual[seasonality:], _naive_forecasting(actual, seasonality)))
    else:
        abs_err = np.abs(_error(actual, predicted))
        abs_err_bench = np.abs(_error(actual, benchmark))

    return abs_err / (abs_err + abs_err_bench + EPSILON)


def _geometric_mean(a, axis=0, dtype=None):
    """ Geometric mean """
    if not isinstance(a, np.ndarray):  # if not an ndarray object attempt to convert it
        log_a = np.log(np.array(a, dtype=dtype))
    elif dtype:  # Must change the default dtype allowing array type
        if isinstance(a, np.ma.MaskedArray):
            log_a = np.log(np.ma.asarray(a, dtype=dtype))
        else:
            log_a = np.log(np.asarray(a, dtype=dtype))
    else:
        log_a = np.log(a)
    return np.exp(log_a.mean(axis=axis))


def mse(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Squared Error """
    return np.mean(np.square(_error(actual, predicted)))


def rmse(actual: np.ndarray, predicted: np.ndarray):
    """ Root Mean Squared Error """
    return np.sqrt(mse(actual, predicted))


def nrmse(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Root Mean Squared Error """
    return rmse(actual, predicted) / (actual.max() - actual.min())


def me(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Error """
    return np.mean(_error(actual, predicted))

def r2_scr(actual:np.ndarray, predicted: np.ndarray):
    """Calculating the r2 value: perasonian correlation coefficient"""
    return metrics.r2_score(actual, predicted)



def mae(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Absolute Error """
    return np.mean(np.abs(_error(actual, predicted)))


mad = mae  # Mean Absolute Deviation (it is the same as MAE)


def gmae(actual: np.ndarray, predicted: np.ndarray):
    """ Geometric Mean Absolute Error """
    return _geometric_mean(np.abs(_error(actual, predicted)))


def mdae(actual: np.ndarray, predicted: np.ndarray):
    """ Median Absolute Error """
    return np.median(np.abs(_error(actual, predicted)))


def mpe(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Percentage Error """
    return np.mean(_percentage_error(actual, predicted))


def mape(actual: np.ndarray, predicted: np.ndarray):
    """
    Mean Absolute Percentage Error
    Properties:
        + Easy to interpret
        + Scale independent
        - Biased, not symmetric
        - Undefined when actual[t] == 0
    Note: result is NOT multiplied by 100
    """
    return np.mean(np.abs(_percentage_error(actual, predicted)))


def mdape(actual: np.ndarray, predicted: np.ndarray):
    """
    Median Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.median(np.abs(_percentage_error(actual, predicted)))


def smape(actual: np.ndarray, predicted: np.ndarray):
    """
    Symmetric Mean Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.mean(2.0 * np.abs(actual - predicted) / ((np.abs(actual) + np.abs(predicted)) + EPSILON))


def smdape(actual: np.ndarray, predicted: np.ndarray):
    """
    Symmetric Median Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.median(2.0 * np.abs(actual - predicted) / ((np.abs(actual) + np.abs(predicted)) + EPSILON))


def maape(actual: np.ndarray, predicted: np.ndarray):
    """
    Mean Arctangent Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.mean(np.arctan(np.abs((actual - predicted) / (actual + EPSILON))))


def mase(actual: np.ndarray, predicted: np.ndarray, seasonality: int = 1):
    """
    Mean Absolute Scaled Error
    Baseline (benchmark) is computed with naive forecasting (shifted by @seasonality)
    """
    return mae(actual, predicted) / mae(actual[seasonality:], _naive_forecasting(actual, seasonality))


def std_ae(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Absolute Error """
    __mae = mae(actual, predicted)
    return np.sqrt(np.sum(np.square(_error(actual, predicted) - __mae))/(len(actual) - 1))


def std_ape(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Absolute Percentage Error """
    __mape = mape(actual, predicted)
    return np.sqrt(np.sum(np.square(_percentage_error(actual, predicted) - __mape))/(len(actual) - 1))


def rmspe(actual: np.ndarray, predicted: np.ndarray):
    """
    Root Mean Squared Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.sqrt(np.mean(np.square(_percentage_error(actual, predicted))))


def rmdspe(actual: np.ndarray, predicted: np.ndarray):
    """
    Root Median Squared Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.sqrt(np.median(np.square(_percentage_error(actual, predicted))))


def rmsse(actual: np.ndarray, predicted: np.ndarray, seasonality: int = 1):
    """ Root Mean Squared Scaled Error """
    q = np.abs(_error(actual, predicted)) / mae(actual[seasonality:], _naive_forecasting(actual, seasonality))
    return np.sqrt(np.mean(np.square(q)))


def inrse(actual: np.ndarray, predicted: np.ndarray):
    """ Integral Normalized Root Squared Error """
    return np.sqrt(np.sum(np.square(_error(actual, predicted))) / np.sum(np.square(actual - np.mean(actual))))


def rrse(actual: np.ndarray, predicted: np.ndarray):
    """ Root Relative Squared Error """
    return np.sqrt(np.sum(np.square(actual - predicted)) / np.sum(np.square(actual - np.mean(actual))))


def mre(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Mean Relative Error """
    return np.mean(_relative_error(actual, predicted, benchmark))


def rae(actual: np.ndarray, predicted: np.ndarray):
    """ Relative Absolute Error (aka Approximation Error) """
    return np.sum(np.abs(actual - predicted)) / (np.sum(np.abs(actual - np.mean(actual))) + EPSILON)


def mrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Mean Relative Absolute Error """
    return np.mean(np.abs(_relative_error(actual, predicted, benchmark)))


def mdrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Median Relative Absolute Error """
    return np.median(np.abs(_relative_error(actual, predicted, benchmark)))


def gmrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Geometric Mean Relative Absolute Error """
    return _geometric_mean(np.abs(_relative_error(actual, predicted, benchmark)))


def mbrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Mean Bounded Relative Absolute Error """
    return np.mean(_bounded_relative_error(actual, predicted, benchmark))


def umbrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Unscaled Mean Bounded Relative Absolute Error """
    __mbrae = mbrae(actual, predicted, benchmark)
    return __mbrae / (1 - __mbrae)


def mda(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Directional Accuracy """
    return np.mean((np.sign(actual[1:] - actual[:-1]) == np.sign(predicted[1:] - predicted[:-1])).astype(int))

# https://www.kaggle.com/carlolepelaars/understanding-the-metric-rmsle
def rmsle(y_true:np.ndarray, y_pred:np.ndarray):
    """
        The Root Mean Squared Log Error (RMSLE) metric

        :param y_true: The ground truth labels given in the dataset/actual values
        :param y_pred: Our predictions/forecasts
        :return: The RMSLE score --> Evaluation metric
    """
    return np.sqrt(mean_squared_log_error(y_true, y_pred))


METRICS = {
    'mse': mse,
    'rmse': rmse,
    'nrmse': nrmse,
    'me': me,
    'mae': mae,
    'mad': mad,
    'gmae': gmae,
    'mdae': mdae,
    'mpe': mpe,
    'mape': mape,
    'mdape': mdape,
    'smape': smape,
    'smdape': smdape,
    'maape': maape,
    'mase': mase,
    'std_ae': std_ae,
    'std_ape': std_ape,
    'rmspe': rmspe,
    'rmdspe': rmdspe,
    'rmsse': rmsse,
    'inrse': inrse,
    'rrse': rrse,
    'mre': mre,
    'rae': rae,
    'r2_scr':r2_scr,
    'mrae': mrae,
    'mdrae': mdrae,
    'gmrae': gmrae,
    'mbrae': mbrae,
    'umbrae': umbrae,
    'mda': mda,
    'rmsle': rmsle
}


def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('mae', 'mse', 'smape', 'umbrae')):
    results = {}
    for name in metrics:
        try:
            results[name] = METRICS[name](actual, predicted)
        except Exception as err:
            results[name] = np.nan
            print('Unable to compute metric {0}: {1}'.format(name, err))
    return results


def evaluate_all(actual: np.ndarray, predicted: np.ndarray):
    return evaluate(actual, predicted, metrics=set(METRICS.keys()))

# Module for Theil's U1
def theils_u1(f:np.ndarray , y:np.ndarray):
    df = pd.DataFrame({'f_i':f, 'y_i': y})
    df['(f_i - y_i)^2'] = np.square(df['f_i'] - df['y_i'])
    df['y_i^2'] = np.square(df['y_i'])
    df['f_i^2'] = np.square(df['f_i'])
    return (np.sqrt(np.mean(df['(f_i - y_i)^2'])))/(np.sqrt(np.mean(df['y_i^2']))+np.sqrt(np.mean(df['f_i^2'])))

# Module for Theil's U2
def theils_u2(f:int , y:int)->float:
    y = y.values.flatten()
    f = f.values.flatten()
    df = pd.DataFrame({'f_i+1':f, 'y_i+1': y})
    df['y_i'] = df['y_i+1'].shift(periods=1)
    df['numerator'] = np.square((df['f_i+1'] - df['y_i+1']) / df['y_i'])
    df['denominator'] = np.square((df['y_i+1'] - df['y_i']) / df['y_i'])
    df.dropna(inplace=True)
    return np.sqrt(np.sum(df['numerator'])/np.sum(df['denominator']))

For the forecast values across different algorithms , please refer to 'Table_08_09_Forecasts_Wavelet_Variants_12M_24M_BRIC_FEWNet_Comparison.xlsx' at "/FEWNet/dataset"

__Brazil: Forecasts Evaluation:  FEWNet and Its Variants:12M (table 08)__

In [7]:
actual = np.array([4.517457  ,4.559198  ,5.195379  ,6.099479  ,6.759116  ,8.056819  ,
                   8.347072  ,8.994823  ,9.679774 ,10.246209 ,10.672622 ,10.738501])
fewnet = np.array([4.51,
4.97,
5.53,
5.86,
6.29,
7.02,
7.62,
8.06,
8.33,
8.40,
8.30,
8.21])
fewnet_d8 = np.array([5.76,
6.17,
6.14,
5.67,
5.52,
5.81,
6.28,
6.84,
7.07,
6.58,
5.84,
5.54])
fewnet_la8 = np.array([7.16,
6.11,
5.40,
5.36,
5.42,
5.04,
4.56,
4.32,
4.59,
5.87,
7.71,
8.63])
fewnet_c6 = np.array([4.06,
5.16,
5.78,
6.32,
7.35,
8.18,
8.51,
8.62,
8.37,
7.92,
7.89,
8.29])
fewnet_bl14 = np.array([3.10,
4.06,
5.24,
5.96,
6.52,
7.17,
7.33,
6.91,
6.93,
7.14,
7.37,
7.88])

ewnet = np.array([3.89,
3.12,
2.72,
2.74,
3.27,
4.17,
5.06,
5.80,
6.27,
6.09,
5.51,
4.91])

In [8]:
# Generate the values of the different evaluation metrics
def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('rmse',
                                                               'mase',
                                                               'smape',
                                                               'mdrae',
                                                               'mdape'
                                                              )):
 results = {}
 for name in metrics:
     try:
         results[name] = METRICS[name](actual, predicted)
     except Exception as err:
         results[name] = np.nan
         print('Unable to compute metric {0}: {1}'.format(name, err))
 return results

print("Evaluation_metrics:  fewnet - ", evaluate(actual,fewnet))
print("Measuring Theils U1: fewnet - ", theils_u1(actual,fewnet))

print("Evaluation_metrics:  fewnet_d8 - ", evaluate(actual,fewnet_d8))
print("Measuring Theils U1: fewnet_d8 - ", theils_u1(actual,fewnet_d8))

print("Evaluation_metrics:  fewnet_la8 - ", evaluate(actual,fewnet_la8))
print("Measuring Theils U1: fewnet_la8 - ", theils_u1(actual,fewnet_la8))

print("Evaluation_metrics:  fewnet_c6 - ", evaluate(actual,fewnet_c6))
print("Measuring Theils U1: fewnet_c6 - ", theils_u1(actual,fewnet_c6))

print("Evaluation_metrics:  fewnet_bl14 - ", evaluate(actual,fewnet_bl14))
print("Measuring Theils U1: fewnet_bl14 - ", theils_u1(actual,fewnet_bl14))

print("Evaluation_metrics:  ewnet - ", evaluate(actual,ewnet))
print("Measuring Theils U1: ewnet - ", theils_u1(actual,ewnet))

Evaluation_metrics:  fewnet -  {'rmse': 1.3002806259453827, 'mase': 1.8061042085540626, 'smape': 0.1221136210018778, 'mdrae': 1.9706139560391018, 'mdape': 0.09701650481312285}
Measuring Theils U1: fewnet -  0.08556488407153964
Evaluation_metrics:  fewnet_d8 -  {'rmse': 2.7590104949435976, 'mase': 4.161495924156783, 'smape': 0.3194839813275897, 'mdrae': 3.810161602244517, 'mdape': 0.27233234430024233}
Measuring Theils U1: fewnet_d8 -  0.1935079492618716
Evaluation_metrics:  fewnet_la8 -  {'rmse': 3.100362108078493, 'mase': 4.787730577590085, 'smape': 0.38975808108326565, 'mdrae': 7.217006225043733, 'mdape': 0.35729546148656854}
Measuring Theils U1: fewnet_la8 -  0.21951081914145007
Evaluation_metrics:  fewnet_c6 -  {'rmse': 1.3649449022612297, 'mase': 1.7655872685249192, 'smape': 0.12429622753087259, 'mdrae': 0.9189538824770076, 'mdape': 0.10689569313622904}
Measuring Theils U1: fewnet_c6 -  0.0881492043899428
Evaluation_metrics:  fewnet_bl14 -  {'rmse': 1.932021968786997, 'mase': 2.703

__Brazil: Forecasts Evaluation:  FEWNet and Its Variants:24M (table 09)__

In [9]:
actual = np.array([4.306152,4.191771,4.005114,3.303158,2.399279,1.877727,2.132417,2.305625,2.438465,3.135329,
3.918350,4.311223,4.517457,4.559198,5.195379,6.099479,6.759116,8.056819,8.347072,8.994823
,9.679774,10.246209,10.672622,10.738501])

fewnet = np.array([4.62,
5.45,
5.63,
5.26,
5.02,
5.08,
5.21,
5.20,
5.10,
5.31,
5.91,
6.04,
6.07,
5.98,
6.05,
6.33,
6.75,
6.90,
6.94,
6.97,
6.96,
6.87,
6.58,
6.55])
fewnet_d8 = np.array([5.05,
4.88,
5.11,
5.51,
5.67,
5.77,
5.57,
5.28,
5.17,
5.20,
5.11,
5.12,
5.53,
6.12,
6.79,
7.31,
7.84,
8.37,
8.77,
8.96,
8.97,
8.81,
8.53,
8.45])
fewnet_la8 = np.array([6.86,
7.31,
7.15,
6.88,
6.81,
6.98,
7.15,
7.36,
7.52,
7.68,
7.98,
7.97,
7.89,
7.67,
7.83,
7.89,
7.68,
7.45,
6.98,
6.15,
5.40,
4.73,
4.08,
3.69])
fewnet_c6 = np.array([6.65,
6.22,
6.22,
6.20,
6.44,
6.64,
6.97,
7.52,
8.21,
8.52,
8.41,
8.41,
8.60,
8.90,
9.09,
9.12,
8.90,
8.92,
9.11,
9.35,
9.24,
9.22,
9.53,
9.73])
fewnet_bl14 = np.array([5.42,
5.03,
4.63,
4.64,
4.58,
4.75,
4.88,
4.94,
4.84,
4.61,
4.44,
4.55,
4.91,
5.36,
5.36,
4.71,
4.48,
4.85,
5.19,
5.30,
5.44,
5.16,
4.85,
5.11])

ewnet = np.array([3.35,
3.38,
3.43,
3.15,
3.32,
3.87,
4.30,
4.68,
5.10,
5.31,
5.72,
6.08,
6.33,
6.57,
7.00,
6.85,
6.14,
5.90,
5.89,
5.68,
5.27,
5.53,
5.50,
5.08])

In [10]:
# Generate the values of the different evaluation metrics
def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('rmse',
                                                               'mase',
                                                               'smape',
                                                               'mdrae',
                                                               'mdape'
                                                              )):
 results = {}
 for name in metrics:
     try:
         results[name] = METRICS[name](actual, predicted)
     except Exception as err:
         results[name] = np.nan
         print('Unable to compute metric {0}: {1}'.format(name, err))
 return results

print("Evaluation_metrics:  fewnet - ", evaluate(actual,fewnet))
print("Measuring Theils U1: fewnet - ", theils_u1(actual,fewnet))

print("Evaluation_metrics:  fewnet_d8 - ", evaluate(actual,fewnet_d8))
print("Measuring Theils U1: fewnet_d8 - ", theils_u1(actual,fewnet_d8))

print("Evaluation_metrics:  fewnet_la8 - ", evaluate(actual,fewnet_la8))
print("Measuring Theils U1: fewnet_la8 - ", theils_u1(actual,fewnet_la8))

print("Evaluation_metrics:  fewnet_c6 - ", evaluate(actual,fewnet_c6))
print("Measuring Theils U1: fewnet_c6 - ", theils_u1(actual,fewnet_c6))

print("Evaluation_metrics:  fewnet_bl14 - ", evaluate(actual,fewnet_bl14))
print("Measuring Theils U1: fewnet_bl14 - ", theils_u1(actual,fewnet_bl14))

print("Evaluation_metrics:  ewnet - ", evaluate(actual,ewnet))
print("Measuring Theils U1: ewnet - ", theils_u1(actual,ewnet))

Evaluation_metrics:  fewnet -  {'rmse': 2.3058873400998925, 'mase': 4.120430728374381, 'smape': 0.38953078535588476, 'mdrae': 4.847743174110954, 'mdape': 0.36357276869052085}
Measuring Theils U1: fewnet -  0.18890182850675324
Evaluation_metrics:  fewnet_d8 -  {'rmse': 1.9228669994913736, 'mase': 3.3041330663347623, 'smape': 0.33653298155626193, 'mdrae': 3.14384662331312, 'mdape': 0.21862590789021652}
Measuring Theils U1: fewnet_d8 -  0.14829178436291274
Evaluation_metrics:  fewnet_la8 -  {'rmse': 4.06081541178201, 'mase': 7.590010604383889, 'smape': 0.6370930365942538, 'mdrae': 9.312874641598466, 'mdape': 0.713103199729614}
Measuring Theils U1: fewnet_la8 -  0.30778597447352574
Evaluation_metrics:  fewnet_c6 -  {'rmse': 3.422905119349102, 'mase': 6.040997383871079, 'smape': 0.5196958514579461, 'mdrae': 6.121875691647211, 'mdape': 0.6513231055113323}
Measuring Theils U1: fewnet_c6 -  0.23655559302275422
Evaluation_metrics:  fewnet_bl14 -  {'rmse': 2.822269129217297, 'mase': 4.6559943897

__Russia: Forecasts Evaluation:  FEWNet and Its Variants:12M (table 08)__

In [11]:
actual = np.array([4.912471 ,5.194607 ,5.666425 ,5.782023 ,5.519744 ,6.014352 ,
                   6.511526 ,6.469069 ,6.692744 ,7.408086,8.135334 ,8.403766])

fewnet = np.array([5.44,
6.38,
6.83,
6.78,
6.51,
6.14,
5.84,
5.80,
6.05,
6.50,
7.02,
7.51])

fewnet_d8 = np.array([3.92,
3.88,
3.83,
4.30,
4.21,
3.76,
5.11,
4.79,
4.58,
4.12,
3.95,
3.86])
fewnet_la8 = np.array([5.83,
6.34,
6.67,
6.70,
6.72,
6.91,
7.19,
7.34,
7.26,
7.26,
7.38,
7.85])
fewnet_c6 = np.array([6.10,
5.83,
6.11,
6.16,
6.24,
6.23,
6.40,
6.70,
6.97,
7.27,
7.49,
7.63])
fewnet_bl14 = np.array([9.87,
10.39,
10.96,
11.55,
12.15,
12.70,
13.09,
12.54,
13.83,
12.90,
13.13,
12.87])


ewnet = np.array([4.74,
5.17,
5.33,
5.46,
5.26,
5.11,
4.81,
4.29,
3.87,
3.66,
3.58,
3.62])

In [12]:
# Generate the values of the different evaluation metrics
def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('rmse',
                                                               'mase',
                                                               'smape',
                                                               'mdrae',
                                                               'mdape'
                                                              )):
 results = {}
 for name in metrics:
     try:
         results[name] = METRICS[name](actual, predicted)
     except Exception as err:
         results[name] = np.nan
         print('Unable to compute metric {0}: {1}'.format(name, err))
 return results

print("Evaluation_metrics:  fewnet - ", evaluate(actual,fewnet))
print("Measuring Theils U1: fewnet - ", theils_u1(actual,fewnet))

print("Evaluation_metrics:  fewnet_d8 - ", evaluate(actual,fewnet_d8))
print("Measuring Theils U1: fewnet_d8 - ", theils_u1(actual,fewnet_d8))

print("Evaluation_metrics:  fewnet_la8 - ", evaluate(actual,fewnet_la8))
print("Measuring Theils U1: fewnet_la8 - ", theils_u1(actual,fewnet_la8))

print("Evaluation_metrics:  fewnet_c6 - ", evaluate(actual,fewnet_c6))
print("Measuring Theils U1: fewnet_c6 - ", theils_u1(actual,fewnet_c6))

print("Evaluation_metrics:  fewnet_bl14 - ", evaluate(actual,fewnet_bl14))
print("Measuring Theils U1: fewnet_bl14 - ", theils_u1(actual,fewnet_bl14))

print("Evaluation_metrics:  ewnet - ", evaluate(actual,ewnet))
print("Measuring Theils U1: ewnet - ", theils_u1(actual,ewnet))

Evaluation_metrics:  fewnet -  {'rmse': 0.8760151621388088, 'mase': 2.210967139399371, 'smape': 0.12882850218506564, 'mdrae': 2.873562086565973, 'mdape': 0.114983026253067}
Measuring Theils U1: fewnet -  0.06788030956309019
Evaluation_metrics:  fewnet_d8 -  {'rmse': 2.4715235893331906, 'mase': 5.9013679026387065, 'smape': 0.40367543513839466, 'mdrae': 4.993705178452615, 'mdape': 0.28761511061398237}
Measuring Theils U1: fewnet_d8 -  0.2311242288813333
Evaluation_metrics:  fewnet_la8 -  {'rmse': 0.8507923655611181, 'mase': 2.1580612236686467, 'smape': 0.12550997608775818, 'mdrae': 2.1270383914714914, 'mdape': 0.1417742454511733}
Measuring Theils U1: fewnet_la8 -  0.0632356078713144
Evaluation_metrics:  fewnet_c6 -  {'rmse': 0.569775039786537, 'mase': 1.2869553224230172, 'smape': 0.07651317704610315, 'mdrae': 1.2395484514409056, 'mdape': 0.07182617220215355}
Measuring Theils U1: fewnet_c6 -  0.04349264541669336
Evaluation_metrics:  fewnet_bl14 -  {'rmse': 5.828428043732182, 'mase': 15.48

__Russia: Forecasts Evaluation:  FEWNet and Its Variants:24M (table 09)__

In [13]:
actual = np.array([3.046190,2.423893,2.311721,2.546287,3.098436,3.026511,3.211885,3.366394,3.573624,3.666989,3.977587,
4.423442,4.912471,5.194607,5.666425,5.782023,5.519744,6.014352,6.511526,6.469069,6.692744,7.408086,
8.135334,8.403766])

fewnet = np.array([4.34,
4.45,
4.25,
3.82,
3.36,
3.17,
3.28,
3.02,
3.05,
3.39,
4.00,
4.67,
5.47,
6.19,
7.03,
7.89,
8.85,
9.52,
9.89,
10.44,
10.76,
10.97,
11.31,
11.73])
fewnet_d8 = np.array([8.02,
7.71,
7.36,
6.67,
5.58,
4.53,
3.52,
2.78,
2.57,
2.25,
1.83,
1.55,
1.71,
1.68,
1.21,
0.52,
-0.07,
-0.47,
-0.29,
-0.02,
-0.18,
-0.15,
0.55,
1.44])
fewnet_la8 = np.array([7.22,
5.71,
4.18,
3.11,
2.83,
3.69,
5.37,
7.39,
9.02,
9.70,
9.68,
8.90,
7.99,
7.20,
6.51,
6.03,
6.21,
6.67,
7.50,
8.62,
9.51,
9.56,
8.74,
7.25])
fewnet_c6 = np.array([4.17,
4.57,
4.74,
4.86,
5.14,
5.45,
5.77,
5.94,
5.87,
5.41,
4.70,
3.98,
3.44,
3.03,
2.65,
2.02,
1.34,
0.87,
0.81,
0.92,
1.05,
1.74,
2.65,
3.59])
fewnet_bl14 = np.array([10.09,
8.74,
8.05,
7.76,
7.51,
7.28,
6.62,
5.51,
4.48,
4.22,
5.07,
6.55,
7.54,
7.58,
6.61,
5.85,
6.42,
7.93,
9.63,
11.22,
12.19,
12.33,
12.24,
12.43])

ewnet = np.array([3.11,
2.41,
1.82,
1.37,
1.07,
1.02,
1.55,
1.87,
2.32,
2.74,
3.12,
3.37,
3.88,
4.59,
5.61,
6.86,
8.33,
9.83,
10.83,
11.62,
12.34,
12.85,
13.15,
13.50])

In [14]:
# Generate the values of the different evaluation metrics
def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('rmse',
                                                               'mase',
                                                               'smape',
                                                               'mdrae',
                                                               'mdape'
                                                              )):
 results = {}
 for name in metrics:
     try:
         results[name] = METRICS[name](actual, predicted)
     except Exception as err:
         results[name] = np.nan
         print('Unable to compute metric {0}: {1}'.format(name, err))
 return results

print("Evaluation_metrics:  fewnet - ", evaluate(actual,fewnet))
print("Measuring Theils U1: fewnet - ", theils_u1(actual,fewnet))

print("Evaluation_metrics:  fewnet_d8 - ", evaluate(actual,fewnet_d8))
print("Measuring Theils U1: fewnet_d8 - ", theils_u1(actual,fewnet_d8))

print("Evaluation_metrics:  fewnet_la8 - ", evaluate(actual,fewnet_la8))
print("Measuring Theils U1: fewnet_la8 - ", theils_u1(actual,fewnet_la8))

print("Evaluation_metrics:  fewnet_c6 - ", evaluate(actual,fewnet_c6))
print("Measuring Theils U1: fewnet_c6 - ", theils_u1(actual,fewnet_c6))

print("Evaluation_metrics:  fewnet_bl14 - ", evaluate(actual,fewnet_bl14))
print("Measuring Theils U1: fewnet_bl14 - ", theils_u1(actual,fewnet_bl14))

print("Evaluation_metrics:  ewnet - ", evaluate(actual,ewnet))
print("Measuring Theils U1: ewnet - ", theils_u1(actual,ewnet))

Evaluation_metrics:  fewnet -  {'rmse': 2.2390790267841525, 'mase': 5.2799124105939335, 'smape': 0.27485726243480463, 'mdrae': 3.5280609339013544, 'mdape': 0.3774030491789776}
Measuring Theils U1: fewnet -  0.1821866427816819
Evaluation_metrics:  fewnet_d8 -  {'rmse': 4.839600187414039, 'mase': 12.963478475435798, 'smape': 1.156573666807049, 'mdrae': 12.457137688045064, 'mdape': 0.8693571650975004}
Measuring Theils U1: fewnet_d8 -  0.5500974264335647
Evaluation_metrics:  fewnet_la8 -  {'rmse': 2.9371482760808427, 'mase': 7.086674071840077, 'smape': 0.40245017668590183, 'mdrae': 6.293142123208815, 'mdape': 0.35927377859404874}
Measuring Theils U1: fewnet_la8 -  0.2359065160567663
Evaluation_metrics:  fewnet_c6 -  {'rmse': 3.5585909549780137, 'mase': 9.534747057851915, 'smape': 0.7678827556013997, 'mdrae': 11.081291313477154, 'mdape': 0.7157477862927318}
Measuring Theils U1: fewnet_c6 -  0.39247399406238975
Evaluation_metrics:  fewnet_bl14 -  {'rmse': 3.8078629763317378, 'mase': 9.920721

__India: Forecasts Evaluation:  FEWNet and Its Variants:12M (table 08)__

In [15]:
actual = np.array([3.686636 ,3.162966 ,4.494493 ,5.665658 ,5.139860 ,
                   5.257646 ,5.577841 ,5.263880 ,4.811442 ,4.403048
                   ,4.518828 ,4.837364])

fewnet = np.array([4.37,
5.04,
5.83,
6.55,
6.04,
5.41,
4.80,
4.40,
4.26,
4.56,
4.57,
5.22])

fewnet_d8 = np.array([3.83,
4.88,
4.79,
3.57,
3.79,
3.93,
3.22,
3.33,
4.17,
5.38,
5.57,
5.91])
fewnet_la8 = np.array([7.61,
6.40,
7.38,
7.78,
7.20,
8.31,
8.79,
8.28,
7.31,
7.15,
6.20,
5.80])
fewnet_c6 = np.array([7.47,
7.09,
6.98,
7.61,
7.27,
7.04,
5.73,
4.93,
4.88,
5.42,
6.01,
6.99])
fewnet_bl14 = np.array([7.58,
8.53,
8.70,
8.58,
7.69,
6.65,
7.03,
7.71,
7.60,
8.90,
9.13,
9.16])

ewnet = np.array([5.35,
5.77,
5.98,
6.46,
6.62,
6.70,
6.89,
6.71,
6.67,
6.35,
6.62,
7.13])

In [16]:
# Generate the values of the different evaluation metrics
def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('rmse',
                                                               'mase',
                                                               'smape',
                                                               'mdrae',
                                                               'mdape'
                                                              )):
 results = {}
 for name in metrics:
     try:
         results[name] = METRICS[name](actual, predicted)
     except Exception as err:
         results[name] = np.nan
         print('Unable to compute metric {0}: {1}'.format(name, err))
 return results

print("Evaluation_metrics:  fewnet - ", evaluate(actual,fewnet))
print("Measuring Theils U1: fewnet - ", theils_u1(actual,fewnet))

print("Evaluation_metrics:  fewnet_d8 - ", evaluate(actual,fewnet_d8))
print("Measuring Theils U1: fewnet_d8 - ", theils_u1(actual,fewnet_d8))

print("Evaluation_metrics:  fewnet_la8 - ", evaluate(actual,fewnet_la8))
print("Measuring Theils U1: fewnet_la8 - ", theils_u1(actual,fewnet_la8))

print("Evaluation_metrics:  fewnet_c6 - ", evaluate(actual,fewnet_c6))
print("Measuring Theils U1: fewnet_c6 - ", theils_u1(actual,fewnet_c6))

print("Evaluation_metrics:  fewnet_bl14 - ", evaluate(actual,fewnet_bl14))
print("Measuring Theils U1: fewnet_bl14 - ", theils_u1(actual,fewnet_bl14))

print("Evaluation_metrics:  ewnet - ", evaluate(actual,ewnet))
print("Measuring Theils U1: ewnet - ", theils_u1(actual,ewnet))

Evaluation_metrics:  fewnet -  {'rmse': 0.8764712793359709, 'mase': 1.4106547603101607, 'smape': 0.14941794694082147, 'mdrae': 1.2188233528613492, 'mdape': 0.1477700608601335}
Measuring Theils U1: fewnet -  0.08831043610424189
Evaluation_metrics:  fewnet_d8 -  {'rmse': 1.4128379065439531, 'mase': 2.4496273608072503, 'smape': 0.2758779399835714, 'mdrae': 3.2788473663335393, 'mdape': 0.24256884238803833}
Measuring Theils U1: fewnet_d8 -  0.15291790328132734
Evaluation_metrics:  fewnet_la8 -  {'rmse': 2.7261893233658894, 'mase': 5.138987632272178, 'smape': 0.43591127916976324, 'mdrae': 6.181438693486631, 'mdape': 0.574431369420648}
Measuring Theils U1: fewnet_la8 -  0.2236620289111868
Evaluation_metrics:  fewnet_c6 -  {'rmse': 2.1493969643059194, 'mase': 3.481851408670804, 'smape': 0.313704084741366, 'mdrae': 2.4901247330984573, 'mdape': 0.3410912730381348}
Measuring Theils U1: fewnet_c6 -  0.1900402326101532
Evaluation_metrics:  fewnet_bl14 -  {'rmse': 3.5896131444536143, 'mase': 6.62058

__India: Forecasts Evaluation:  FEWNet and Its Variants:24M (table 09)__

In [17]:
actual = np.array([9.634551,7.491857,6.840391,5.501618,5.448718,5.095541,5.063291,5.329154,5.625000,5.636574,5.902162,
5.284787,3.686636,3.162966,4.494493,5.665658,5.139860,5.257646,5.577841,5.263880,4.811442,4.403048,
4.518828,4.837364])

fewnet = np.array([7.69,
6.34,
5.92,
5.11,
4.53,
4.15,
3.95,
5.57,
5.37,
4.25,
3.85,
3.37,
3.86,
4.75,
5.39,
5.65,
4.78,
4.36,
4.87,
4.06,
4.28,
4.73,
5.04,
4.72])
fewnet_d8 = np.array([4.74,
4.17,
4.28,
4.44,
4.79,
5.17,
5.48,
4.91,
4.94,
4.93,
4.96,
4.56,
3.89,
4.02,
3.48,
2.14,
1.72,
1.45,
1.25,
2.07,
2.74,
3.13,
2.49,
2.87])
fewnet_la8 = np.array([9.69,
9.86,
9.42,
8.67,
8.50,
7.92,
8.10,
8.35,
8.82,
9.06,
9.06,
9.31,
8.49,
8.18,
9.17,
10.27,
10.67,
10.82,
9.60,
8.46,
7.61,
7.06,
6.66,
5.70])
fewnet_c6 = np.array([11.10,
9.91,
9.32,
8.28,
8.16,
8.53,
8.77,
8.06,
7.68,
8.39,
7.73,
6.14,
5.31,
5.18,
5.22,
5.70,
5.26,
4.92,
3.95,
3.85,
4.24,
4.57,
5.00,
6.09])
fewnet_bl14 = np.array([6.88,
6.22,
6.45,
6.43,
8.02,
8.40,
7.83,
7.04,
6.71,
6.41,
7.46,
7.91,
7.40,
6.36,
6.00,
7.41,
8.12,
8.87,
9.71,
9.52,
9.98,
11.59,
12.41,
12.92])

ewnet = np.array([8.69,
8.08,
7.74,
7.17,
6.37,
6.24,
6.60,
7.44,
6.64,
5.91,
4.99,
3.81,
3.76,
3.96,
4.96,
6.01,
6.22,
6.63,
6.46,
6.26,
7.90,
9.49,
10.56,
10.90])

In [18]:
# Generate the values of the different evaluation metrics
def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('rmse',
                                                               'mase',
                                                               'smape',
                                                               'mdrae',
                                                               'mdape'
                                                              )):
 results = {}
 for name in metrics:
     try:
         results[name] = METRICS[name](actual, predicted)
     except Exception as err:
         results[name] = np.nan
         print('Unable to compute metric {0}: {1}'.format(name, err))
 return results

print("Evaluation_metrics:  fewnet - ", evaluate(actual,fewnet))
print("Measuring Theils U1: fewnet - ", theils_u1(actual,fewnet))

print("Evaluation_metrics:  fewnet_d8 - ", evaluate(actual,fewnet_d8))
print("Measuring Theils U1: fewnet_d8 - ", theils_u1(actual,fewnet_d8))

print("Evaluation_metrics:  fewnet_la8 - ", evaluate(actual,fewnet_la8))
print("Measuring Theils U1: fewnet_la8 - ", theils_u1(actual,fewnet_la8))

print("Evaluation_metrics:  fewnet_c6 - ", evaluate(actual,fewnet_c6))
print("Measuring Theils U1: fewnet_c6 - ", theils_u1(actual,fewnet_c6))

print("Evaluation_metrics:  fewnet_bl14 - ", evaluate(actual,fewnet_bl14))
print("Measuring Theils U1: fewnet_bl14 - ", theils_u1(actual,fewnet_bl14))

print("Evaluation_metrics:  ewnet - ", evaluate(actual,ewnet))
print("Measuring Theils U1: ewnet - ", theils_u1(actual,ewnet))

Evaluation_metrics:  fewnet -  {'rmse': 1.0392446855079414, 'mase': 1.4908116430106715, 'smape': 0.17022205025935655, 'mdrae': 1.4127997472489435, 'mdape': 0.1441501293546582}
Measuring Theils U1: fewnet -  0.09910524186920876
Evaluation_metrics:  fewnet_d8 -  {'rmse': 2.3193250193802433, 'mase': 3.199764385236634, 'smape': 0.43792613165808186, 'mdrae': 3.547456962081331, 'mdape': 0.28004388524067203}
Measuring Theils U1: fewnet_d8 -  0.24545711222669825
Evaluation_metrics:  fewnet_la8 -  {'rmse': 3.5676782787301833, 'mase': 5.780960507573575, 'smape': 0.47848657714979087, 'mdrae': 9.580525905547486, 'mdape': 0.5906982304946435}
Measuring Theils U1: fewnet_la8 -  0.24863326991900958
Evaluation_metrics:  fewnet_c6 -  {'rmse': 1.9605168858466466, 'mase': 2.868738434228687, 'smape': 0.2637598015461407, 'mdrae': 3.8517272335348065, 'mdape': 0.3007651320251644}
Measuring Theils U1: fewnet_c6 -  0.15606632952307453
Evaluation_metrics:  fewnet_bl14 -  {'rmse': 3.773984297532671, 'mase': 5.450

__China: Forecasts Evaluation:  FEWNet and Its Variants:12M (table 08)__

In [19]:
actual = np.array([0.2697842 ,-0.2216312 ,-0.4227999  ,0.3219466  ,0.9226485  ,
                   1.6443494  ,1.2401969  ,1.9125597  ,0.6415800  ,0.4613803  ,
                   1.4328405  ,2.4772978])
fewnet = np.array([-0.19,
-0.45,
-0.17,
0.50,
1.24,
2.03,
2.44,
3.15,
2.86,
3.34,
4.45,
5.44])
fewnet_d8 = np.array([4.71,
5.09,
5.36,
5.58,
5.39,
4.76,
5.12,
4.47,
4.25,
4.13,
4.25,
4.34])
fewnet_la8 = np.array([4.26,
3.20,
2.15,
1.35,
1.85,
1.35,
1.71,
1.95,
2.08,
1.78,
1.47,
1.59])
fewnet_c6 = np.array([2.05,
2.00,
2.57,
2.72,
3.55,
4.22,
4.15,
3.94,
3.45,
3.45,
3.03,
3.20])
fewnet_bl14 = np.array([3.31,
4.43,
4.21,
4.52,
4.65,
5.54,
5.91,
6.41,
7.14,
6.69,
6.17,
6.10])

ewnet = np.array([-0.50,
-0.63,
-0.58,
-0.42,
-0.41,
0.04,
0.60,
0.95,
1.35,
1.58,
1.83,
2.36])

In [20]:
# Generate the values of the different evaluation metrics
def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('rmse',
                                                               'mase',
                                                               'smape',
                                                               'mdrae',
                                                               'mdape'
                                                              )):
 results = {}
 for name in metrics:
     try:
         results[name] = METRICS[name](actual, predicted)
     except Exception as err:
         results[name] = np.nan
         print('Unable to compute metric {0}: {1}'.format(name, err))
 return results

print("Evaluation_metrics:  fewnet - ", evaluate(actual,fewnet))
print("Measuring Theils U1: fewnet - ", theils_u1(actual,fewnet))

print("Evaluation_metrics:  fewnet_d8 - ", evaluate(actual,fewnet_d8))
print("Measuring Theils U1: fewnet_d8 - ", theils_u1(actual,fewnet_d8))

print("Evaluation_metrics:  fewnet_la8 - ", evaluate(actual,fewnet_la8))
print("Measuring Theils U1: fewnet_la8 - ", theils_u1(actual,fewnet_la8))

print("Evaluation_metrics:  fewnet_c6 - ", evaluate(actual,fewnet_c6))
print("Measuring Theils U1: fewnet_c6 - ", theils_u1(actual,fewnet_c6))

print("Evaluation_metrics:  fewnet_bl14 - ", evaluate(actual,fewnet_bl14))
print("Measuring Theils U1: fewnet_bl14 - ", theils_u1(actual,fewnet_bl14))

print("Evaluation_metrics:  ewnet - ", evaluate(actual,ewnet))
print("Measuring Theils U1: ewnet - ", theils_u1(actual,ewnet))

Evaluation_metrics:  fewnet -  {'rmse': 1.6997079014910703, 'mase': 1.9248904821556483, 'smape': 0.8471877905494606, 'mdrae': 1.745440938336422, 'mdape': 0.9989147895399106}
Measuring Theils U1: fewnet -  0.42768239319136137
Evaluation_metrics:  fewnet_d8 -  {'rmse': 4.063662512630428, 'mase': 5.870235681210722, 'smape': 1.3821821542201025, 'mdrae': 7.060192158397496, 'mdape': 5.233074678529414}
Measuring Theils U1: fewnet_d8 -  0.6732525179804933
Evaluation_metrics:  fewnet_la8 -  {'rmse': 1.850018077494878, 'mase': 2.0613219326039647, 'smape': 0.9075430067793188, 'mdrae': 1.1624401682935128, 'mdape': 1.6235468017827959}
Measuring Theils U1: fewnet_la8 -  0.5376625419323505
Evaluation_metrics:  fewnet_c6 -  {'rmse': 2.394769170308037, 'mase': 3.4704222769721675, 'smape': 1.234044504784243, 'mdrae': 3.5688615597446436, 'mdape': 3.6124843882843747}
Measuring Theils U1: fewnet_c6 -  0.5325009004411808
Evaluation_metrics:  fewnet_bl14 -  {'rmse': 4.633754439460999, 'mase': 6.8279018043456

__China: Forecasts Evaluation:  FEWNet and Its Variants:24M (table 09)__

In [19]:
actual = np.array([4.4131455,5.4205607,5.1803885,4.2711235,3.2467532,2.4118738,2.5069638,1.7576318,2.3875115,
1.7304189,0.5415162,-0.4496403,0.2697842,-0.2216312,-0.4227999,0.3219466,0.9226485,1.6443494,
1.2401969,1.9125597,0.6415800,0.4613803,1.4328405,2.4772978])

fewnet = np.array([4.10,
3.71,
3.68,
3.25,
3.28,
3.31,
3.58,
3.63,
3.40,
3.39,
3.24,
2.90,
3.20,
3.75,
3.38,
3.42,
3.06,
2.82,
2.58,
2.39,
2.44,
2.16,
1.88,
2.08])

fewnet_d8 = np.array([1.50,
1.79,
2.04,
2.03,
1.78,
2.95,
3.46,
3.09,
2.84,
1.86,
1.86,
1.80,
2.35,
3.32,
4.10,
4.07,
4.83,
3.65,
2.97,
2.95,
2.70,
3.51,
3.40,
3.64])
fewnet_la8 = np.array([5.51,
5.75,
5.63,
6.67,
5.80,
4.88,
4.27,
3.95,
4.17,
4.35,
4.60,
4.43,
4.11,
3.04,
2.83,
1.28,
1.60,
1.87,
2.49,
2.80,
2.69,
2.82,
2.59,
2.82])
fewnet_bl14 = np.array([4.45,
4.03,
3.24,
3.13,
3.42,
3.85,
2.98,
2.59,
1.93,
1.84,
2.45,
2.30,
3.11,
3.16,
3.27,
2.88,
2.03,
0.30,
0.49,
0.04,
0.66,
0.04,
-1.02,
-0.96])
fewnet_c6 = np.array([4.03,
4.10,
3.87,
3.60,
3.84,
2.63,
2.73,
2.73,
2.91,
3.29,
3.09,
3.21,
3.03,
3.03,
3.41,
3.90,
3.08,
3.61,
3.19,
3.17,
3.30,
3.43,
4.45,
4.81])

ewnet = np.array([4.60,
5.00,
5.11,
5.16,
5.01,
5.28,
5.41,
5.55,
5.83,
5.85,
5.78,
5.74,
5.55,
5.30,
5.28,
5.30,
5.25,
5.27,
4.93,
4.68,
4.62,
4.33,
4.21,
4.13])

In [20]:
# Generate the values of the different evaluation metrics
def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('rmse',
                                                               'mase',
                                                               'smape',
                                                               'mdrae',
                                                               'mdape'
                                                              )):
 results = {}
 for name in metrics:
     try:
         results[name] = METRICS[name](actual, predicted)
     except Exception as err:
         results[name] = np.nan
         print('Unable to compute metric {0}: {1}'.format(name, err))
 return results

print("Evaluation_metrics:  fewnet - ", evaluate(actual,fewnet))
print("Measuring Theils U1: fewnet - ", theils_u1(actual,fewnet))

print("Evaluation_metrics:  fewnet_d8 - ", evaluate(actual,fewnet_d8))
print("Measuring Theils U1: fewnet_d8 - ", theils_u1(actual,fewnet_d8))

print("Evaluation_metrics:  fewnet_la8 - ", evaluate(actual,fewnet_la8))
print("Measuring Theils U1: fewnet_la8 - ", theils_u1(actual,fewnet_la8))

print("Evaluation_metrics:  fewnet_c6 - ", evaluate(actual,fewnet_c6))
print("Measuring Theils U1: fewnet_c6 - ", theils_u1(actual,fewnet_c6))

print("Evaluation_metrics:  fewnet_bl14 - ", evaluate(actual,fewnet_bl14))
print("Measuring Theils U1: fewnet_bl14 - ", theils_u1(actual,fewnet_bl14))

print("Evaluation_metrics:  ewnet - ", evaluate(actual,ewnet))
print("Measuring Theils U1: ewnet - ", theils_u1(actual,ewnet))

Evaluation_metrics:  fewnet -  {'rmse': 2.009707674291087, 'mase': 2.3689195232667446, 'smape': 0.8179678670901619, 'mdrae': 2.4987164571243077, 'mdape': 0.8370135961149536}
Measuring Theils U1: fewnet -  0.3566292141790691
Evaluation_metrics:  fewnet_d8 -  {'rmse': 2.4339840429368382, 'mase': 2.99952609583184, 'smape': 0.9663012317787967, 'mdrae': 2.464763847994232, 'mdape': 0.9888851396370112}
Measuring Theils U1: fewnet_d8 -  0.445899999536152
Evaluation_metrics:  fewnet_la8 -  {'rmse': 2.3190734979953715, 'mase': 2.7460890333110206, 'smape': 0.8478966832438455, 'mdrae': 2.8298871986460448, 'mdape': 0.907671984565878}
Measuring Theils U1: fewnet_la8 -  0.35548124271894105
Evaluation_metrics:  fewnet_c6 -  {'rmse': 2.2178837575350565, 'mase': 2.6792963709891118, 'smape': 0.8721919186325326, 'mdrae': 2.373457105189354, 'mdape': 1.0685144414389822}
Measuring Theils U1: fewnet_c6 -  0.3726926379802187
Evaluation_metrics:  fewnet_bl14 -  {'rmse': 1.89556556020127, 'mase': 2.1409981430828

In [23]:
# Convert into HTML
%%shell
jupyter nbconvert --to html /content/Table_08_09_Evaluation_Module_Comparison_FEWNet_Variants.ipynb

[NbConvertApp] Converting notebook /content/Table_08_09_Evaluation_Module_Comparison_FEWNet_Variants.ipynb to html
[NbConvertApp] Writing 784085 bytes to /content/Table_08_09_Evaluation_Module_Comparison_FEWNet_Variants.html
